In [1]:
from eurostat_dataset import eurostat_dataset # packed in object the method to wrangle eurostat's table 
import pandas as pd
import sqlite3
import plotly.express as px

In [22]:
# !!You don't need this here, just import the 'eurostat_dataset.py'!!!


# def get_codes(dictionary):
#       url_dict = "https://ec.europa.eu/eurostat/" + \
#           "estat-navtree-portlet-prod/BulkDownloadListing" + \
#           "?sort=1&downfile=dic%2Fen%2F" +\
#           dictionary + ".dic"
#       with request.urlopen(url_dict) as resp:
#                 file_content = resp.readlines()
#       d={}
#       for i in file_content:
#             if len(i) > 1:
#                   row=i.decode().split('\t')
#                   d[row[0]] = row[1].strip()
#       return d

The first is to process the Final energy comsumption data

In [4]:
# full information from network

# get_renew = get_df(dataset='NRG_BAL_C') # Final energy comsumption
# 
# Now it is better to use this code:
# get_renew=eurostat_dataset(code='NRG_BAL_C').get_df()

In [2]:
dataset_info=eurostat_dataset(code='NRG_BAL_S').DatasetInfo()
dataset_info

RemoteDisconnected: Remote end closed connection without response

In [ ]:
# I won't be able download 'NRG_BAL_C'. It is huge!!

In [5]:
# uncheck this comment to download dataset to the sqlite database
# eurostat_dataset(code='NRG_BAL_S').WriteToDatabase() 
# write to database, so you don't need to wait everytime when you restart the kernel or the file

In [22]:
# eurostat_dataset(code='NRG_INF_EPCRW').WriteToDatabase()

In [4]:
#check the tables we have written to the database:
def db_tables():
    con = sqlite3.connect('ngr.db')
    cursor = con.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cursor.fetchall())
    con.close

In [5]:
# read table from db to DataFrame
def read_df_from_db(table):
    con = sqlite3.connect('ngr.db')
    cursor = con.cursor()   
    df = pd.read_sql_query("SELECT * FROM "+ table , con)
    con.close
    df.drop(columns=['index'],inplace=True)
    #df.value = pd.to_numeric(df.value)
    return df

In [6]:
db_tables() # we can see the tables in th database

[('nrg_bal_s',), ('nrg_inf_epcrw',)]


In [7]:
balance = read_df_from_db('nrg_bal_s')
balance = balance.query('unit=="Gigawatt-hour" and siec=="Total" and nrg_bal=="Final consumption - energy use"')
get_energy = read_df_from_db('nrg_inf_epcrw')
#.query('unit=="Gigawatt-hour"')


In [52]:
balance.head()

,nrg_bal,siec,unit,geo,date,value
5778,Final consumption - energy use,Total,Gigawatt-hour,Albania,2020,21474.051
5779,Final consumption - energy use,Total,Gigawatt-hour,Austria,2020,288624.871
5780,Final consumption - energy use,Total,Gigawatt-hour,Bosnia and Herzegovina,2020,46804.808
5781,Final consumption - energy use,Total,Gigawatt-hour,Belgium,2020,360985.528
5782,Final consumption - energy use,Total,Gigawatt-hour,Bulgaria,2020,110633.154


In [49]:
get_energy.unit.unique() #in the table balance we have Gigawatt-hour

array(['Megawatt'], dtype=object)

In [8]:
balance.value=pd.to_numeric(balance.value)
balance.date=pd.to_numeric(balance.date)

In [9]:
fig = px.scatter(balance, x="date", y="value", color="geo")
fig.show()

In [6]:
# transformation into DF
# df_get_renew = pd.DataFrame(get_renew)
# df_get_renew
# It is a huge dataset and I cannot download all data, so I am working with 'Simplified energy balances'

In [30]:
df_get_renew.shape

(1224180, 35)

In [29]:
df_get_renew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224180 entries, 0 to 1224179
Data columns (total 35 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   nrg_bal  1224180 non-null  object 
 1   siec     1224180 non-null  object 
 2   unit     1224180 non-null  object 
 3   geo      1224180 non-null  object 
 4   2020     653130 non-null   float64
 5   2019     669060 non-null   float64
 6   2018     669060 non-null   float64
 7   2017     669060 non-null   float64
 8   2016     669060 non-null   float64
 9   2015     669060 non-null   float64
 10  2014     669060 non-null   float64
 11  2013     653130 non-null   float64
 12  2012     637200 non-null   float64
 13  2011     637200 non-null   float64
 14  2010     637200 non-null   float64
 15  2009     621270 non-null   float64
 16  2008     621270 non-null   float64
 17  2007     621270 non-null   float64
 18  2006     621270 non-null   float64
 19  2005     621270 non-null   float64
 20  20

All the further steps are taken to get final energy consumption for all countries from 2012 to 2020 in GWh  measurement unit

In [37]:
print(df_get_renew['nrg_bal'].unique())

['Available for final consumption' 'Distribution losses' 'Exports'
 'Final consumption - energy use'
 'Final consumption - industry sector - construction - energy use'
 'Final consumption - industry sector - chemical and petrochemical - energy use'
 'Final consumption - industry sector - energy use'
 'Final consumption - industry sector - food, beverages and tobacco - energy use'
 'Final consumption - industry sector - iron and steel - energy use'
 'Final consumption - industry sector - machinery - energy use'
 'Final consumption - industry sector - mining and quarrying - energy use'
 'Final consumption - industry sector - non-energy use'
 'Final consumption - industry sector - non-ferrous metals - energy use'
 'Final consumption - industry sector - non-metallic minerals - energy use'
 'Final consumption - industry sector - not elsewhere specified - energy use'
 'Final consumption - industry sector - paper, pulp and printing - energy use'
 'Final consumption - industry sector - transpo

In [32]:
df_get_renew["nrg_bal_obj"] = df_get_renew["nrg_bal"].astype(str)
df_get_renew.head()

,nrg_bal,siec,unit,geo,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,nrg_bal_obj
0,Available for final consumption,Bioenergy,Gigawatt-hour,Albania,3013.181,3211.056,3129.617,2804.720,3049.733,2728.383,...,3192.500,3344.722,3504.167,3671.389,3846.389,4029.722,4221.944,4221.944,4221.944,Available for final consumption
1,Available for final consumption,Bioenergy,Gigawatt-hour,Austria,54591.852,55571.298,55689.684,58228.039,57876.168,57507.590,...,27778.728,28863.769,29117.247,27703.651,25186.310,27367.890,27110.141,27905.111,25744.633,Available for final consumption
2,Available for final consumption,Bioenergy,Gigawatt-hour,Bosnia and Herzegovina,14750.870,13990.576,13379.486,4843.579,5936.294,5937.175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Available for final consumption
3,Available for final consumption,Bioenergy,Gigawatt-hour,Belgium,27827.872,25799.034,27791.685,27165.203,26994.596,25412.611,...,4792.250,4198.739,4464.628,4344.736,3076.269,3139.375,3958.904,4108.185,4034.103,Available for final consumption
4,Available for final consumption,Bioenergy,Gigawatt-hour,Bulgaria,18159.810,16781.094,16364.406,14661.761,14428.844,13753.572,...,4759.722,2776.667,2724.166,2394.221,1865.723,1656.454,1818.057,1333.333,2005.833,Available for final consumption


In [36]:
df_get_renew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224180 entries, 0 to 1224179
Data columns (total 36 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   nrg_bal      1224180 non-null  object 
 1   siec         1224180 non-null  object 
 2   unit         1224180 non-null  object 
 3   geo          1224180 non-null  object 
 4   2020         653130 non-null   float64
 5   2019         669060 non-null   float64
 6   2018         669060 non-null   float64
 7   2017         669060 non-null   float64
 8   2016         669060 non-null   float64
 9   2015         669060 non-null   float64
 10  2014         669060 non-null   float64
 11  2013         653130 non-null   float64
 12  2012         637200 non-null   float64
 13  2011         637200 non-null   float64
 14  2010         637200 non-null   float64
 15  2009         621270 non-null   float64
 16  2008         621270 non-null   float64
 17  2007         621270 non-null   float64
 18  20

In [40]:
df_get_renew_fincon = df_get_renew.loc[df_get_renew['nrg_bal'] == 'Final consumption - energy use']
df_get_renew_fincon

,nrg_bal,siec,unit,geo,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,nrg_bal_obj
27120,Final consumption - energy use,Bioenergy,Gigawatt-hour,Albania,3080.264,3211.056,3129.617,2804.720,3049.733,2728.383,...,3192.500,3344.722,3504.167,3671.389,3846.389,4029.722,4221.944,4221.944,4221.944,Final consumption - energy use
27121,Final consumption - energy use,Bioenergy,Gigawatt-hour,Austria,54617.316,55603.307,55714.824,58250.872,57903.829,57534.736,...,27778.698,28863.741,29117.257,27703.637,25186.305,27367.955,27110.124,27905.100,25744.602,Final consumption - energy use
27122,Final consumption - energy use,Bioenergy,Gigawatt-hour,Bosnia and Herzegovina,14750.870,13990.576,13379.208,4843.852,5936.572,5919.490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Final consumption - energy use
27123,Final consumption - energy use,Bioenergy,Gigawatt-hour,Belgium,28321.853,26215.905,28111.988,27509.906,27338.839,25723.481,...,4792.250,4198.739,4464.628,4344.736,3076.269,3139.375,3958.904,4108.185,4034.103,Final consumption - energy use
27124,Final consumption - energy use,Bioenergy,Gigawatt-hour,Bulgaria,18258.865,17015.287,16340.454,14731.810,14439.100,13761.236,...,4751.944,2723.611,2490.833,2277.554,1795.168,1560.620,1746.946,1333.333,2005.833,Final consumption - energy use
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36139,Final consumption - energy use,Non-renewable municipal waste,Terajoule,Slovakia,515.000,440.000,8.000,419.000,71.000,550.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,Final consumption - energy use
36140,Final consumption - energy use,Non-renewable municipal waste,Terajoule,Türkiye,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,Final consumption - energy use
36141,Final consumption - energy use,Non-renewable municipal waste,Terajoule,Ukraine,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,Final consumption - energy use
36142,Final consumption - energy use,Non-renewable municipal waste,Terajoule,United Kingdom,NaN,5475.000,5504.000,5275.000,4193.000,4709.000,...,286.000,186.000,673.000,656.000,605.000,525.000,531.000,571.000,536.000,Final consumption - energy use


In [41]:
print(df_get_renew_fincon['siec'].unique())

['Bioenergy' 'Solid fossil fuels' 'Anthracite' 'Coking coal'
 'Other bituminous coal' 'Sub-bituminous coal' 'Lignite' 'Coke oven coke'
 'Gas coke' 'Patent fuel' 'Brown coal briquettes' 'Coal tar'
 'Coke oven gas' 'Manufactured gases' 'Gas works gas' 'Blast furnace gas'
 'Other recovered gases' 'Electricity' 'Fossil energy' 'Natural gas'
 'Heat' 'Nuclear heat'
 'Oil and petroleum products (excluding biofuel portion)' 'Crude oil'
 'Natural gas liquids' 'Refinery feedstocks'
 'Additives and oxygenates (excluding biofuel portion)'
 'Other hydrocarbons' 'Refinery gas' 'Ethane' 'Liquefied petroleum gases'
 'Naphtha' 'Aviation gasoline'
 'Motor gasoline (excluding biofuel portion)' 'Gasoline-type jet fuel'
 'Kerosene-type jet fuel (excluding biofuel portion)' 'Other kerosene'
 'Gas oil and diesel oil (excluding biofuel portion)' 'Fuel oil'
 'White spirit and special boiling point industrial spirits' 'Lubricants'
 'Paraffin waxes' 'Petroleum coke' 'Bitumen' 'Other oil products n.e.c.'
 'Peat a

In [42]:
df_get_renew_fincon_total = df_get_renew_fincon.loc[df_get_renew_fincon['siec'] == 'Total']
df_get_renew_fincon_total

,nrg_bal,siec,unit,geo,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,nrg_bal_obj
35514,Final consumption - energy use,Total,Gigawatt-hour,Albania,21474.051,23876.813,24820.884,24177.936,22091.860,22823.850,...,10719.609,9611.654,11928.655,11037.049,10912.369,10922.459,11889.507,16371.327,22265.844,Final consumption - energy use
35515,Final consumption - energy use,Total,Gigawatt-hour,Austria,288624.871,305027.230,302830.312,308232.060,303542.303,296290.399,...,250415.878,243008.558,248958.967,229092.039,218885.178,224114.636,220061.112,225737.683,208914.111,Final consumption - energy use
35516,Final consumption - energy use,Total,Gigawatt-hour,Bosnia and Herzegovina,46804.808,48935.410,48984.752,40680.956,41215.947,38575.626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Final consumption - energy use
35517,Final consumption - energy use,Total,Gigawatt-hour,Belgium,360985.528,378903.241,385929.337,383395.441,388756.658,385533.434,...,387543.245,379346.273,387137.413,363532.757,353682.831,347129.988,350916.864,348200.881,330464.804,Final consumption - energy use
35518,Final consumption - energy use,Total,Gigawatt-hour,Bulgaria,110633.154,112889.753,113452.073,113344.852,110693.751,109188.868,...,115074.279,118430.761,128131.965,124760.205,118611.468,118097.784,121937.584,138556.553,185601.676,Final consumption - energy use
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35635,Final consumption - energy use,Total,Terajoule,Slovakia,402389.827,429080.035,414975.319,414619.929,386263.255,374286.417,...,424205.646,431833.322,431327.049,416147.139,413354.335,427741.246,467297.483,494195.555,579625.615,Final consumption - energy use
35636,Final consumption - energy use,Total,Terajoule,Türkiye,4234655.450,4149124.802,4129566.302,4206687.475,3781439.427,3636109.261,...,2057017.821,2075910.541,2013489.806,1849464.822,1681540.137,1755022.630,1643811.562,1594908.427,1579015.317,Final consumption - energy use
35637,Final consumption - energy use,Total,Terajoule,Ukraine,1849905.636,1963041.095,2027153.668,1982273.606,2044254.858,1988233.954,...,3142668.197,3453304.235,3634193.392,3805233.320,3864933.924,4547103.552,5413258.133,6058287.220,6016449.244,Final consumption - energy use
35638,Final consumption - energy use,Total,Terajoule,United Kingdom,NaN,5083381.556,5132333.922,5063965.885,5103126.882,5026811.971,...,5720050.390,5689514.395,5823053.560,5512475.275,5532421.580,5536860.610,5449510.400,5552469.776,5308638.346,Final consumption - energy use


In [43]:
df_get_renew_fincon_total2 = df_get_renew_fincon_total.loc[df_get_renew_fincon_total['unit'] == 'Gigawatt-hour']
df_get_renew_fincon_total2

,nrg_bal,siec,unit,geo,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,nrg_bal_obj
35514,Final consumption - energy use,Total,Gigawatt-hour,Albania,2.147405e+04,2.387681e+04,2.482088e+04,2.417794e+04,2.209186e+04,2.282385e+04,...,1.071961e+04,9.611654e+03,1.192866e+04,1.103705e+04,1.091237e+04,1.092246e+04,1.188951e+04,1.637133e+04,22265.844,Final consumption - energy use
35515,Final consumption - energy use,Total,Gigawatt-hour,Austria,2.886249e+05,3.050272e+05,3.028303e+05,3.082321e+05,3.035423e+05,2.962904e+05,...,2.504159e+05,2.430086e+05,2.489590e+05,2.290920e+05,2.188852e+05,2.241146e+05,2.200611e+05,2.257377e+05,208914.111,Final consumption - energy use
35516,Final consumption - energy use,Total,Gigawatt-hour,Bosnia and Herzegovina,4.680481e+04,4.893541e+04,4.898475e+04,4.068096e+04,4.121595e+04,3.857563e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Final consumption - energy use
35517,Final consumption - energy use,Total,Gigawatt-hour,Belgium,3.609855e+05,3.789032e+05,3.859293e+05,3.833954e+05,3.887567e+05,3.855334e+05,...,3.875432e+05,3.793463e+05,3.871374e+05,3.635328e+05,3.536828e+05,3.471300e+05,3.509169e+05,3.482009e+05,330464.804,Final consumption - energy use
35518,Final consumption - energy use,Total,Gigawatt-hour,Bulgaria,1.106332e+05,1.128898e+05,1.134521e+05,1.133449e+05,1.106938e+05,1.091889e+05,...,1.150743e+05,1.184308e+05,1.281320e+05,1.247602e+05,1.186115e+05,1.180978e+05,1.219376e+05,1.385566e+05,185601.676,Final consumption - energy use
35519,Final consumption - energy use,Total,Gigawatt-hour,Cyprus,1.777713e+04,1.891797e+04,1.839021e+04,1.804448e+04,1.733575e+04,1.656022e+04,...,1.486154e+04,1.420454e+04,1.413870e+04,1.339741e+04,1.283463e+04,1.236588e+04,1.201241e+04,1.154712e+04,9920.896,Final consumption - energy use
35520,Final consumption - energy use,Total,Gigawatt-hour,Czechia,2.762517e+05,2.817057e+05,2.812842e+05,2.842431e+05,2.755215e+05,2.685820e+05,...,2.812574e+05,2.922413e+05,2.960390e+05,2.913721e+05,2.946954e+05,3.146788e+05,3.225512e+05,3.264464e+05,362561.850,Final consumption - energy use
35521,Final consumption - energy use,Total,Gigawatt-hour,Germany (until 1990 former territory of the FRG),2.251765e+06,2.335354e+06,2.333962e+06,2.378478e+06,2.368647e+06,2.326364e+06,...,2.490197e+06,2.508348e+06,2.560695e+06,2.449186e+06,2.410843e+06,2.467317e+06,2.447989e+06,2.526766e+06,2543117.158,Final consumption - energy use
35522,Final consumption - energy use,Total,Gigawatt-hour,Denmark,1.527109e+05,1.575483e+05,1.605762e+05,1.606873e+05,1.600788e+05,1.565707e+05,...,1.675543e+05,1.684460e+05,1.725896e+05,1.659242e+05,1.625817e+05,1.628754e+05,1.572489e+05,1.596496e+05,150321.500,Final consumption - energy use
35523,Final consumption - energy use,Total,Gigawatt-hour,Euro area - 19 countries (from 2015),8.026523e+06,8.591978e+06,8.630465e+06,8.650930e+06,8.580933e+06,8.445910e+06,...,8.590815e+06,8.438200e+06,8.510953e+06,8.156643e+06,7.993518e+06,8.067753e+06,8.081861e+06,8.237822e+06,8038571.044,Final consumption - energy use


In [69]:
df_get_renew_fincon_total_last = df_get_renew_fincon_total2.loc['35514':'35555','geo':'2012']
df_get_renew_fincon_total_last

,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012
35514,Albania,2.147405e+04,2.387681e+04,2.482088e+04,2.417794e+04,2.209186e+04,2.282385e+04,2.394122e+04,2.281655e+04,2.082494e+04
35515,Austria,2.886249e+05,3.050272e+05,3.028303e+05,3.082321e+05,3.035423e+05,2.962904e+05,2.879859e+05,3.000607e+05,2.931221e+05
35516,Bosnia and Herzegovina,4.680481e+04,4.893541e+04,4.898475e+04,4.068096e+04,4.121595e+04,3.857563e+04,3.609515e+04,NaN,NaN
35517,Belgium,3.609855e+05,3.789032e+05,3.859293e+05,3.833954e+05,3.887567e+05,3.855334e+05,3.672835e+05,3.961131e+05,3.838927e+05
35518,Bulgaria,1.106332e+05,1.128898e+05,1.134521e+05,1.133449e+05,1.106938e+05,1.091889e+05,1.032919e+05,1.009572e+05,1.058722e+05
35519,Cyprus,1.777713e+04,1.891797e+04,1.839021e+04,1.804448e+04,1.733575e+04,1.656022e+04,1.601855e+04,1.596165e+04,1.736125e+04
35520,Czechia,2.762517e+05,2.817057e+05,2.812842e+05,2.842431e+05,2.755215e+05,2.685820e+05,2.609707e+05,2.689594e+05,2.712320e+05
35521,Germany (until 1990 former territory of the FRG),2.251765e+06,2.335354e+06,2.333962e+06,2.378478e+06,2.368647e+06,2.326364e+06,2.293773e+06,2.419708e+06,2.356965e+06
35522,Denmark,1.527109e+05,1.575483e+05,1.605762e+05,1.606873e+05,1.600788e+05,1.565707e+05,1.504057e+05,1.565516e+05,1.583135e+05
35523,Euro area - 19 countries (from 2015),8.026523e+06,8.591978e+06,8.630465e+06,8.650930e+06,8.580933e+06,8.445910e+06,8.279002e+06,8.704211e+06,8.682039e+06


In [76]:
df_get_renew_fincon_total_last = df_get_renew_fincon_total_last.reset_index(drop=True)
df_get_renew_fincon_total_last

,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012
0,Albania,2.147405e+04,2.387681e+04,2.482088e+04,2.417794e+04,2.209186e+04,2.282385e+04,2.394122e+04,2.281655e+04,2.082494e+04
1,Austria,2.886249e+05,3.050272e+05,3.028303e+05,3.082321e+05,3.035423e+05,2.962904e+05,2.879859e+05,3.000607e+05,2.931221e+05
2,Bosnia and Herzegovina,4.680481e+04,4.893541e+04,4.898475e+04,4.068096e+04,4.121595e+04,3.857563e+04,3.609515e+04,NaN,NaN
3,Belgium,3.609855e+05,3.789032e+05,3.859293e+05,3.833954e+05,3.887567e+05,3.855334e+05,3.672835e+05,3.961131e+05,3.838927e+05
4,Bulgaria,1.106332e+05,1.128898e+05,1.134521e+05,1.133449e+05,1.106938e+05,1.091889e+05,1.032919e+05,1.009572e+05,1.058722e+05
5,Cyprus,1.777713e+04,1.891797e+04,1.839021e+04,1.804448e+04,1.733575e+04,1.656022e+04,1.601855e+04,1.596165e+04,1.736125e+04
6,Czechia,2.762517e+05,2.817057e+05,2.812842e+05,2.842431e+05,2.755215e+05,2.685820e+05,2.609707e+05,2.689594e+05,2.712320e+05
7,Germany (until 1990 former territory of the FRG),2.251765e+06,2.335354e+06,2.333962e+06,2.378478e+06,2.368647e+06,2.326364e+06,2.293773e+06,2.419708e+06,2.356965e+06
8,Denmark,1.527109e+05,1.575483e+05,1.605762e+05,1.606873e+05,1.600788e+05,1.565707e+05,1.504057e+05,1.565516e+05,1.583135e+05
9,Euro area - 19 countries (from 2015),8.026523e+06,8.591978e+06,8.630465e+06,8.650930e+06,8.580933e+06,8.445910e+06,8.279002e+06,8.704211e+06,8.682039e+06


In [78]:
df_get_renew_fincon_total_last.set_index('geo', inplace=True) 
df_get_renew_fincon_total_last

,2020,2019,2018,2017,2016,2015,2014,2013,2012
geo,,,,,,,,,
Albania,2.147405e+04,2.387681e+04,2.482088e+04,2.417794e+04,2.209186e+04,2.282385e+04,2.394122e+04,2.281655e+04,2.082494e+04
Austria,2.886249e+05,3.050272e+05,3.028303e+05,3.082321e+05,3.035423e+05,2.962904e+05,2.879859e+05,3.000607e+05,2.931221e+05
Bosnia and Herzegovina,4.680481e+04,4.893541e+04,4.898475e+04,4.068096e+04,4.121595e+04,3.857563e+04,3.609515e+04,NaN,NaN
Belgium,3.609855e+05,3.789032e+05,3.859293e+05,3.833954e+05,3.887567e+05,3.855334e+05,3.672835e+05,3.961131e+05,3.838927e+05
Bulgaria,1.106332e+05,1.128898e+05,1.134521e+05,1.133449e+05,1.106938e+05,1.091889e+05,1.032919e+05,1.009572e+05,1.058722e+05
Cyprus,1.777713e+04,1.891797e+04,1.839021e+04,1.804448e+04,1.733575e+04,1.656022e+04,1.601855e+04,1.596165e+04,1.736125e+04
Czechia,2.762517e+05,2.817057e+05,2.812842e+05,2.842431e+05,2.755215e+05,2.685820e+05,2.609707e+05,2.689594e+05,2.712320e+05
Germany (until 1990 former territory of the FRG),2.251765e+06,2.335354e+06,2.333962e+06,2.378478e+06,2.368647e+06,2.326364e+06,2.293773e+06,2.419708e+06,2.356965e+06
Denmark,1.527109e+05,1.575483e+05,1.605762e+05,1.606873e+05,1.600788e+05,1.565707e+05,1.504057e+05,1.565516e+05,1.583135e+05


In [79]:
# this is the final table I got for futher visualisation
df_get_renew_fincon_total_last.T

geo,Albania,Austria,Bosnia and Herzegovina,Belgium,Bulgaria,Cyprus,Czechia,Germany (until 1990 former territory of the FRG),Denmark,Euro area - 19 countries (from 2015),...,Portugal,Romania,Serbia,Sweden,Slovenia,Slovakia,Türkiye,Ukraine,United Kingdom,Kosovo (under United Nations Security Council Resolution 1244/99)
2020,21474.051,288624.871,46804.808,360985.528,110633.154,17777.130,276251.734,2251765.205,152710.872,8026522.519,...,176952.427,272983.777,100696.470,365813.706,51507.402,111774.952,1176293.181,513862.677,NaN,17731.951
2019,23876.813,305027.230,48935.410,378903.241,112889.753,18917.965,281705.716,2335354.497,157548.312,8591977.931,...,190224.237,275778.159,97238.474,364075.095,56554.593,119188.899,1152534.667,545289.193,1412050.432,17620.974
2018,24820.884,302830.312,48984.752,385929.337,113452.073,18390.206,281284.245,2333961.821,160576.170,8630465.209,...,187876.587,272662.921,98099.457,369010.578,57600.223,115270.922,1147101.750,563098.241,1425648.312,17108.031
2017,24177.936,308232.060,40680.956,383395.441,113344.852,18044.478,284243.065,2378477.963,160687.340,8650930.437,...,184925.111,267259.190,97264.464,370413.727,57672.936,115172.203,1168524.298,550631.557,1406657.190,17631.823
2016,22091.860,303542.303,41215.947,388756.658,110693.751,17335.750,275521.455,2368646.992,160078.788,8580932.636,...,181901.796,255106.956,95440.788,372598.234,56913.244,107295.349,1050399.841,567848.572,1417535.245,16666.215
2015,22823.850,296290.399,38575.626,385533.434,109188.868,16560.224,268582.033,2326363.971,156570.676,8445910.093,...,180856.675,251210.797,91712.848,366472.330,54838.275,103968.449,1010030.350,552287.209,1396336.659,15583.361
2014,23941.215,287985.855,36095.152,367283.461,103291.851,16018.554,260970.747,2293773.490,150405.746,8279001.860,...,178943.690,249700.374,89337.149,359702.233,53591.224,101651.892,940219.127,673305.177,1355988.652,14313.046
2013,22816.554,300060.664,NaN,396113.064,100957.206,15961.653,268959.413,2419707.803,156551.560,8704211.429,...,173383.735,251572.314,95385.555,368340.459,55873.193,109463.001,912568.698,757625.493,1435088.648,13578.934
2012,20824.945,293122.137,NaN,383892.650,105872.176,17361.253,271232.039,2356965.292,158313.544,8682038.597,...,175747.771,263177.218,97834.969,375783.591,57101.178,106978.148,920612.405,772279.393,1426519.457,14181.294


In [80]:
df_get_renew_fincon_total_last.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, Albania to Kosovo (under United Nations Security Council Resolution 1244/99)
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2020    41 non-null     float64
 1   2019    42 non-null     float64
 2   2018    42 non-null     float64
 3   2017    42 non-null     float64
 4   2016    42 non-null     float64
 5   2015    42 non-null     float64
 6   2014    42 non-null     float64
 7   2013    41 non-null     float64
 8   2012    40 non-null     float64
dtypes: float64(9)
memory usage: 4.3+ KB


In [81]:
import seaborn as sb

In [ ]:
# plot dynamics of energy consumption for all the countries

In [ ]:
# probably, heatmap could be useful also for visualisation

The next step is to get Electricity production capacities for renewables and wastes

In [44]:
# the names of the adata frames should be further changed with places  - get_renew for this part and get_energy for the previous one 
get_energy = get_df(dataset='NRG_INF_EPCRW') # Electricity production capacities for renewables and wastes
get_energy

,siec,plant_tec,unit,geo,2020,2019,2018,2017,2016,2015,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Solid biofuels,Net maximum electrical capacity,Megawatt,Albania,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Solid biofuels,Net maximum electrical capacity,Megawatt,Austria,816.199,813.428,824.253,837.771,916.695,938.621,...,787.0,747.0,609.0,607.0,544.0,412.0,412.0,400.0,400.0,400.0
2,Solid biofuels,Net maximum electrical capacity,Megawatt,Bosnia and Herzegovina,1.120,1.220,0.250,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Solid biofuels,Net maximum electrical capacity,Megawatt,Belgium,563.500,539.200,526.300,559.200,566.300,591.300,...,38.0,18.0,11.0,19.0,23.0,11.0,11.0,18.0,26.0,26.0
4,Solid biofuels,Net maximum electrical capacity,Megawatt,Bulgaria,15.064,23.513,33.019,22.973,19.000,34.000,...,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,Municipal waste,Net maximum electrical capacity,Megawatt,Slovakia,12.000,12.000,12.000,12.000,19.000,11.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1071,Municipal waste,Net maximum electrical capacity,Megawatt,Türkiye,4.600,4.499,2.499,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1072,Municipal waste,Net maximum electrical capacity,Megawatt,Ukraine,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1073,Municipal waste,Net maximum electrical capacity,Megawatt,United Kingdom,NaN,1321.434,1136.727,1090.933,1028.285,929.940,...,161.0,162.0,115.0,115.0,87.0,87.0,50.0,45.0,31.0,31.0


In [46]:
df_get_energy = pd.DataFrame(get_energy)
df_get_energy

,siec,plant_tec,unit,geo,2020,2019,2018,2017,2016,2015,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Solid biofuels,Net maximum electrical capacity,Megawatt,Albania,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Solid biofuels,Net maximum electrical capacity,Megawatt,Austria,816.199,813.428,824.253,837.771,916.695,938.621,...,787.0,747.0,609.0,607.0,544.0,412.0,412.0,400.0,400.0,400.0
2,Solid biofuels,Net maximum electrical capacity,Megawatt,Bosnia and Herzegovina,1.120,1.220,0.250,0.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Solid biofuels,Net maximum electrical capacity,Megawatt,Belgium,563.500,539.200,526.300,559.200,566.300,591.300,...,38.0,18.0,11.0,19.0,23.0,11.0,11.0,18.0,26.0,26.0
4,Solid biofuels,Net maximum electrical capacity,Megawatt,Bulgaria,15.064,23.513,33.019,22.973,19.000,34.000,...,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,Municipal waste,Net maximum electrical capacity,Megawatt,Slovakia,12.000,12.000,12.000,12.000,19.000,11.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1071,Municipal waste,Net maximum electrical capacity,Megawatt,Türkiye,4.600,4.499,2.499,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1072,Municipal waste,Net maximum electrical capacity,Megawatt,Ukraine,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1073,Municipal waste,Net maximum electrical capacity,Megawatt,United Kingdom,NaN,1321.434,1136.727,1090.933,1028.285,929.940,...,161.0,162.0,115.0,115.0,87.0,87.0,50.0,45.0,31.0,31.0


In [49]:
df_get_energy.shape

(1075, 35)

In [50]:
df_get_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075 entries, 0 to 1074
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   siec       1075 non-null   object 
 1   plant_tec  1075 non-null   object 
 2   unit       1075 non-null   object 
 3   geo        1075 non-null   object 
 4   2020       1050 non-null   float64
 5   2019       1074 non-null   float64
 6   2018       1075 non-null   float64
 7   2017       1075 non-null   float64
 8   2016       1075 non-null   float64
 9   2015       1075 non-null   float64
 10  2014       1075 non-null   float64
 11  2013       1025 non-null   float64
 12  2012       1000 non-null   float64
 13  2011       1000 non-null   float64
 14  2010       1000 non-null   float64
 15  2009       975 non-null    float64
 16  2008       975 non-null    float64
 17  2007       975 non-null    float64
 18  2006       975 non-null    float64
 19  2005       975 non-null    float64
 20  2004    

In [54]:
print(df_get_energy['siec'].unique())

['Solid biofuels' 'Pure biogasoline' 'Pure biodiesels'
 'Other liquid biofuels' 'Biogases' 'Hydro' 'Pure hydro power'
 'Run-of-river hydro power' 'Mixed hydro power' 'Pumped hydro power'
 'Geothermal' 'Wind' 'Wind on shore' 'Wind off shore' 'Solar'
 'Solar thermal' 'Solar photovoltaic'
 'Solar photovoltaic (20 kW - 1000 kW)' 'Solar photovoltaic (< 20 kW)'
 'Solar photovoltaic (1+ MW)' 'Solar photovoltaic (Off grid)'
 'Tide, wave, ocean' 'Waste' 'Industrial waste (non-renewable)'
 'Municipal waste']


In [52]:
df_get_energy_fincon = df_get_energy.loc[df_get_energy['unit'] == 'Gigawatt-hour']
df_get_energy_fincon

,siec,plant_tec,unit,geo,2020,2019,2018,2017,2016,2015,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
